---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [15]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    cur_state = None
    sf_list = list()
    with open('university_towns.txt') as f:
        for line in f:
            if '[edit]' in line:
                state = line.split('[')[0]
                cur_state = state
            else:
                region = line.split('(')[0].strip()
                sf_list.append([cur_state,region])
    return pd.DataFrame(sf_list,columns=["State", "RegionName"])
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_gdp():
    gdp = pd.read_excel('gdplev.xls',
                       skiprows = list(range(0,5))+[6,7] ,parse_cols = "E:G")
    gdp.rename(columns={"Unnamed: 0":"Quarter"}, inplace= True)
    gdp = gdp[gdp['Quarter']>="2000q1"]
    
    for index in gdp.index:
        if index-1 not in gdp.index:
            gdp.loc[index,"GDP_Growth"] = 0
        else:
            gdp.loc[index,"GDP_Growth"] = gdp.loc[index]["GDP in billions of chained 2009 dollars"] - gdp.loc[index-1]["GDP in billions of chained 2009 dollars"]
    return gdp
get_gdp()

,Quarter,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,GDP_Growth
212,2000q1,10031.0,12359.1,0.0
213,2000q2,10278.3,12592.5,233.4
214,2000q3,10357.4,12607.7,15.2
215,2000q4,10472.3,12679.3,71.6
216,2001q1,10508.1,12643.3,-36.0
217,2001q2,10638.4,12710.3,67.0
218,2001q3,10639.5,12670.1,-40.2
219,2001q4,10701.3,12705.3,35.2
220,2002q1,10834.4,12822.3,117.0
221,2002q2,10934.8,12893.0,70.7


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    r_start_list = list()
    for index in gdp.index[:-1]:
        if gdp.loc[index]['GDP_Growth'] < 0 and gdp.loc[index+1]['GDP_Growth'] < 0:
            return gdp.loc[index]['Quarter']
get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    r_start_list = list()
    for index in gdp.index[:-3]:
        if gdp.loc[index]['GDP_Growth'] <0  and gdp.loc[index+1]['GDP_Growth']<0 and \
        gdp.loc[index+2]['GDP_Growth'] >0 and gdp.loc[index+3]['GDP_Growth'] >0:
            return gdp.loc[index+3]['Quarter']
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    r_start_list = list()
    for index in gdp.index[:-3]:
        if gdp.loc[index]['GDP_Growth'] < 0  and gdp.loc[index+1]['GDP_Growth']< 0 and \
        gdp.loc[index+2]['GDP_Growth'] > 0 and gdp.loc[index+3]['GDP_Growth'] > 0:
            return gdp.loc[index+1]['Quarter']
get_recession_bottom()

'2009q2'

In [32]:
def get_clean_housing_data():
    df_states = pd.DataFrame.from_dict(states,orient='index')
    df_states.reset_index(inplace=True)
    df_states.rename(columns={0:"StateName", "index":"State"}, inplace=True)
    
    df_housing = pd.read_csv("City_Zhvi_AllHomes.csv")
    selected_cols = ["RegionName","State" ]
    year = 2000
    while year <= 2016:
        for month in range(1, 13):
            if month <= 9:
                cur_month = str(year)+ "-0" + str(month)
            else:
                cur_month = str(year)+ "-" + str(month)
            if cur_month in df_housing:
                selected_cols.append(cur_month)
        year += 1
    df_housing = df_housing[selected_cols]
    df = df_housing.merge(df_states, left_on='State', right_on='State', how='inner')
    df.drop('State', axis=1, inplace=True)
    df.rename(columns={"StateName":"State"}, inplace = True)
    df.set_index(["State","RegionName"], inplace=True)
    return df
get_clean_housing_data()

2000-01   2000-02   2000-03   2000-04  \
State    RegionName                                                        
New York New York                      NaN       NaN       NaN       NaN   
         Rochester                 53300.0   53600.0   53800.0   53600.0   
         Yonkers                  219800.0  222200.0  224700.0  227200.0   
         Syracuse                  56200.0   56800.0   57400.0   57900.0   
         Town of Greece            95100.0   95100.0   95100.0   95200.0   
         Albany                    91000.0   91800.0   91900.0   92200.0   
         Town of Poughkeepsie     130700.0  132000.0  133200.0  134300.0   
         New Rochelle             360200.0  362400.0  365700.0  367700.0   
         Town of Colonie          109800.0  110700.0  111300.0  112000.0   
         Mount Vernon             236800.0  239400.0  241800.0  244200.0   
         Utica                     50200.0   50800.0   51100.0   51100.0   
         Valley Stream            220600.0  224000.0  224900.0  224600.0   
         Schenectady               72000.0   72300.0   72500.0   72700.0   
         White Plains             280800.0  283900.0  288300.0  292900.0   
         Town of Hempstead        168700.0  170600.0  171200.0  171500.0   
         Town of Binghamton        81200.0   81700.0   82000.0   82600.0   
         Town of Newburgh         115400.0  115900.0  116700.0  117600.0   
         Troy                      74300.0   74700.0   75200.0   75900.0   
         Niagara Falls             50100.0   50400.0   50400.0   50000.0   
         Irondequoit               83400.0   83400.0   83400.0   83300.0   
         Binghamton                51500.0   51800.0   52000.0   52000.0   
         Massapequa               246300.0  249700.0  250500.0  250400.0   
         Town of Elmira            51000.0   51200.0   51400.0   51400.0   
         Henrietta                 94300.0   94700.0   94700.0   94200.0   
         Town of Webster          125800.0  126300.0  127200.0  127900.0   
         Patchogue                134500.0  134200.0  134500.0  134900.0   
         Clifton Park             140000.0  140700.0  141200.0  141800.0   
         Freeport                 181800.0  184300.0  185400.0  185500.0   
         Lindenhurst              167900.0  169400.0  168900.0  168500.0   
         Levittown                187800.0  190700.0  191400.0  191900.0   
...                                    ...       ...       ...       ...   
Vermont  Plainfield                    NaN       NaN       NaN       NaN   
         West Brattleboro              NaN       NaN       NaN       NaN   
         Duxbury                       NaN       NaN       NaN       NaN   
         Rochester                     NaN       NaN       NaN       NaN   
         Sharon                        NaN       NaN       NaN       NaN   
         White River Junction          NaN       NaN       NaN       NaN   
         Irasburg                      NaN       NaN       NaN       NaN   
         Mt Holly                      NaN       NaN       NaN       NaN   
         Ferrisburgh                   NaN       NaN       NaN       NaN   
         Lincoln                       NaN       NaN       NaN       NaN   
         Franklin                      NaN       NaN       NaN       NaN   
         Bridport                      NaN       NaN       NaN       NaN   
         Fayston                       NaN       NaN       NaN       NaN   
         Tunbridge                     NaN       NaN       NaN       NaN   
         Ryegate                       NaN       NaN       NaN       NaN   
         Shaftsbury                    NaN       NaN       NaN       NaN   
         Shoreham                      NaN       NaN       NaN       NaN   
         Fairlee                       NaN       NaN       NaN       NaN   
         Corinth                       NaN       NaN       NaN       NaN   
         Brookfield                    NaN       NaN       NaN       NaN   
       

In [37]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_housing = get_clean_housing_data()
    year = 2000
    while year <= 2016:
        for quarter in range(1, 5):
            cur_quarter = str(year) + "q" + str(quarter)
            involved_months = [quarter*3 - 2, quarter* 3 -1, quarter*3]
            involved_columns = list()
            for month in involved_months:
                if month < 10:
                    cur_col = str(year) + "-0" + str(month)
                else:
                    cur_col =  str(year) + "-" + str(month)
                if cur_col in df_housing:
                    involved_columns.append(cur_col)
            if len(involved_columns)>0:
                df_housing[cur_quarter] = df_housing[involved_columns].mean(axis=1)
                df_housing.drop(involved_columns, axis=1, inplace=True)
        year += 1
    return df_housing
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State    RegionName                                                             
New York New York                           NaN            NaN            NaN   
         Rochester                 53566.666667   53233.333333   53200.000000   
         Yonkers                  222233.333333  230900.000000  245533.333333   
         Syracuse                  56800.000000   57866.666667   57400.000000   
         Town of Greece            95100.000000   95500.000000   96333.333333   
         Albany                    91566.666667   92100.000000   91233.333333   
         Town of Poughkeepsie     131966.666667  135433.333333  139566.666667   
         New Rochelle             362766.666667  370866.666667  382233.333333   
         Town of Colonie          110600.000000  112400.000000  113166.666667   
         Mount Vernon             239333.333333  247300.000000  258633.333333   
         Utica                     50700.000000   51233.333333   52600.000000   
         Valley Stream            223166.666667  225166.666667  229133.333333   
         Schenectady               72266.666667   73100.000000   73733.333333   
         White Plains             284333.333333  297833.333333  314366.666667   
         Town of Hempstead        170166.666667  172266.666667  176800.000000   
         Town of Binghamton        81633.333333   82500.000000   81700.000000   
         Town of Newburgh         116000.000000  118400.000000  120233.333333   
         Troy                      74733.333333   76533.333333   78866.666667   
         Niagara Falls             50300.000000   50133.333333   50433.333333   
         Irondequoit               83400.000000   83500.000000   84400.000000   
         Binghamton                51766.666667   52000.000000   53366.666667   
         Massapequa               248833.333333  251233.333333  254500.000000   
         Town of Elmira            51200.000000   51600.000000   51700.000000   
         Henrietta                 94566.666667   94400.000000   95766.666667   
         Town of Webster          126433.333333  128600.000000  130833.333333   
         Patchogue                134400.000000  135200.000000  141966.666667   
         Clifton Park             140633.333333  142500.000000  143466.666667   
         Freeport                 183833.333333  186433.333333  190400.000000   
         Lindenhurst              168733.333333  169100.000000  173033.333333   
         Levittown                189966.666667  193366.666667  197866.666667   
...                                         ...            ...            ...   
Vermont  Plainfield                         NaN            NaN            NaN   
         West Brattleboro                   NaN            NaN            NaN   
         Duxbury                            NaN            NaN            NaN   
         Rochester                          NaN            NaN            NaN   
         Sharon                             NaN            NaN            NaN   
         White River Junction               NaN            NaN            NaN   
         Irasburg                           NaN            NaN            NaN   
         Mt Holly                           NaN            NaN            NaN   
         Ferrisburgh                        NaN            NaN            NaN   
         Lincoln                            NaN            NaN            NaN   
         Franklin                           NaN            NaN            NaN   
         Bridport                           NaN            NaN            NaN   
         Fayston                            NaN            NaN            NaN   
         Tunbridge                          NaN            NaN            NaN   
         Ryegate                            NaN            NaN            NaN   
         Shaftsbury                         NaN            NaN            NaN   
         Shoreham                           NaN            NaN          

In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"